In [ ]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

# Find the repository root by searching upward for a 'pyFMM' directory
start = Path.cwd().resolve()
repo_root = None
for p in [start] + list(start.parents):
    if (p / 'pyFMM').is_dir():
        repo_root = str(p)
        break
if repo_root is None:
    raise RuntimeError("Could not find 'pyFMM' in any parent directory of cwd")
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    
from pyFMM import *
import numpy as np

In [ ]:
np.random.seed(42)

N = 10000                              # number of points
q = np.random.choice([-1, 1], size=N)  # source strengths with magnitude 1 and random sign
#q = np.ones(N)                        # source strengths all positive 1
source_area_size = 1.0
#---------- moment order ---------
p = 4
#---------------------------------

In [ ]:
#--------- generate a number of random points in side box ----------
# NOTE - with this we get slight asymmetries compared to spherical distribution
#       this meains higher order moments are more imporant
LLC = np.array([ -1.0, -1.0, -1.0 ]) * source_area_size   # Lower Left Corner
URC = np.array([  1.0,  1.0,  1.0 ]) * source_area_size   # Upper Right Corner
X = np.random.rand(N, 3)             # uniform in [0,1)
X = LLC + (URC - LLC) * X            # uniform in [LLC, URC]
#------------------------------------------------------------------
#--------- convert to polar coordinates ----------------
x_polar = cart_to_sphe(X)
#------------------------------------------------------

In [ ]:
# child centres at the 8 octants
child_centres = np.array([[-0.5, -0.5, -0.5],
                 [ 0.5, -0.5, -0.5],
                 [-0.5,  0.5, -0.5],
                 [ 0.5,  0.5, -0.5],
                 [-0.5, -0.5,  0.5],
                 [ 0.5, -0.5,  0.5],
                 [-0.5,  0.5,  0.5],
                 [ 0.5,  0.5,  0.5]]) * source_area_size

# assign sources to children by sign of coordinates
child_masks = []
for cx in child_centres:
    x_mask = np.logical_and(X[:,0] >= cx[0] - source_area_size *0.5, X[:,0] <= cx[0] + source_area_size * 0.5)   
    y_mask = np.logical_and(X[:,1] >= cx[1] - source_area_size *0.5, X[:,1] <= cx[1] + source_area_size * 0.5)
    z_mask = np.logical_and(X[:,2] >= cx[2] - source_area_size *0.5, X[:,2] <= cx[2] + source_area_size * 0.5)
    mask = np.logical_and(np.logical_and(x_mask, y_mask), z_mask)

    child_masks.append(mask)

In [ ]:
#--------- generate target points outside box ----------
# Points lie in a small cloud centered at a distance 'dist' from origin
Ntgt = N
dist = 10 * source_area_size
center = np.array([1.0, 1.0, 1.0]) * dist
r_max = source_area_size
x_tgt = generate_points_in_sphere(Ntgt, r_max, center=center)
x_tgt_polar = cart_to_sphe(x_tgt)
#------------------------------------------------------

In [ ]:
#--------- generate target points outside box ----------
# Points lie in a "halo" around the box - roughly at a distance size(URC-LLC)*r_margin
#Ntgt = N
#r_min = source_area_size * 10
#r_max = source_area_size * 11
#x_tgt = generate_points_in_sphere(Ntgt, r_max, r_min=r_min)
#x_tgt_polar = cart_to_sphe(x_tgt)
#------------------------------------------------------

In [ ]:
# build one array per child
child_points = [X[mask] for mask in child_masks]

# pick a distinct colour per child
child_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red',
                'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray']

# now call plot_points_3d with *all* children plus the targets
fig, ax = plot_points_3d(
    *child_points, x_tgt,   # unpack each child’s points as its own arg
    labels=[f'Child {i+1}' for i in range(len(child_points))] + ['Targets'],
    colors=child_colors + ['black'],   # black or red for targets
    sizes=[0.5]*len(child_points) + [2.0],
    title='3D Source and Target Distribution (Octants)',
)


In [ ]:

M_all = []
M_trans_all = []
x1 = np.array([0.0, 0.0, 0.0])



for i,cx in enumerate(child_centres):
    s_pts = X[child_masks[i]]
    q_pts = q[child_masks[i]]
    s_rel = s_pts  - cx
    s_rel_polar = cart_to_sphe(s_rel)
    M = P2M_sphe(s_rel_polar, q_pts, p)

    M_trans_all.append(M2M_sphe(M, cx, x1))
    M_all.append(M)
M_trans_all = np.array(M_trans_all) 
M_trans = np.sum(M_trans_all, axis=0)


P_oct_mom = []
P_oct_mom_tran = []
P_oct_dir = []

for i,cx in enumerate(child_centres):
    M = M_all[i]


    s_pts = X[child_masks[i]]
    q_pts = q[child_masks[i]]
    t_pts = x_tgt

    s_rel = s_pts  - cx
    t_rel = t_pts - cx
    t_sphe = cart_to_sphe(t_rel)

    P_dir =  P_direct_cart(s_pts, q_pts, t_pts)
    P_mom = P_sphe(M, t_sphe)
    P_mom_tran = P_sphe( M_trans_all[i], t_sphe)

    P_oct_dir.append(P_dir)
    P_oct_mom.append(P_mom)
    P_oct_mom_tran.append(P_mom_tran)


rel_errs = []
rel_err_trans = []
for i in range(len(child_centres)):
    rel_errors = np.abs(P_oct_dir[i] - P_oct_mom[i]) / np.abs(P_oct_dir[i])
    rel_errs.append(rel_errors)

    rel_errors_tran = np.abs(P_oct_dir[i] - P_oct_mom_tran[i]) / np.abs(P_oct_dir[i])
    rel_err_trans.append(rel_errors_tran)

In [ ]:
# rel_errs is your list: one 1D array per octant
labels = [f"Child {i+1} - {child_centres[i]}" for i in range(len(rel_errs))]
fig, ax = plot_segmented_errors(rel_errs, labels=labels, title="Per-octant relative errors")

In [ ]:
# rel_errs is your list: one 1D array per octant
labels = [f"Child {i+1} - {child_centres[i]}" for i in range(len(rel_err_trans))]
fig, ax = plot_segmented_errors(rel_err_trans, labels=labels, title="Per-octant relative errors translated moments")

In [ ]:
A = np.sum(np.abs(q)) / 8 # to get rough error per octant
rho = np.sqrt(child_centres[0][0]**2 + child_centres[0][1]**2 + child_centres[0][2]**2)
a = source_area_size *0.5
r = np.linalg.norm(center)
err = A / (r - (a + rho)) * ( (a + rho) / r)**(p+1)
print(f"Estimated error bound: {err}")


In [ ]:
#---------- compute moments and potentials ----------
M_dir = P2M_sphe(x_polar, q, p=p)
P_mom_all = P_sphe(M_dir, x_tgt_polar)
P_mom = P_sphe(M_trans, x_tgt_polar)
P_dir_vec =  P_direct_cart(X, q, x_tgt, eps=1e-10)
#------------------------------------------------------

In [ ]:
#--------- comput relative error between direct summation and moment evaluation ----------
relative_errors = np.abs(P_dir_vec - P_mom) / np.abs(P_dir_vec)
relative_errors_all = np.abs(P_dir_vec - P_mom_all) / np.abs(P_dir_vec)

#------------------------------------------------------
fig, ax = plot_ylog(
    relative_errors, relative_errors_all,
    title='Relative Error per Target',
    xlabel='Target Index',
    ylabel='Relative Error (log scale)',
    sizes=[2,2],
    labels=["M_tran", "M_all"],
    legend=True
)
plt.show()

In [ ]:
#--------------- evaluate potential at center point --------------
center = np.array([1.0, 1.0, 1.0]) * dist 
center_sphe = cart_to_sphe(np.array([center]))[0]
P_center_mom = P_sphe(M_trans, np.array([center_sphe]))[0]
P_center_dir =  P_direct_cart(X, q, np.array([center]), eps=1e-10)[0]
#------------------------------------------------------
#-------------- print results --------------
print("Moment eval at center:", P_center_dir)
print("Direct eval at center:", P_center_mom)
print("Diff = ", np.abs(P_center_dir - P_center_mom))

#------------------------------------------------------

In [ ]:
#--------------- compute theoretical error bound for M2M at center point --------------
A = np.sum(np.abs(q))
rho = np.sqrt(child_centres[0][0]**2 + child_centres[0][1]**2 + child_centres[0][2]**2)
a = source_area_size *0.5
r = np.linalg.norm(center)
err = A / (r - (a + rho)) * ( (a + rho) / r)**(p+1)
rel_error_bound = err / np.abs(P_center_dir)
print(f"Estimated error bound: {err}")
print(f"Relative error bound: {rel_error_bound}")
#--------------------------------------------------------------------------------------
#------------------ compare to actual center error ------------------
actual_error = np.abs(P_center_dir - P_center_mom)
rel_actual_error = actual_error / np.abs(P_center_dir)
print("")
print(f"Actual error at center: {actual_error}")
print("")
#--------------------------------------------------------------------
#------------- compare to actual errors over all target points --------------
#------------- compare to actual errors over all targets ----------------
print("Actual error max:", np.max(relative_errors)  , " relative to bound",  np.max(relative_errors)/rel_error_bound * 100, "%")
print("Actual error min:", np.min(relative_errors)  , " relative to bound",  np.min(relative_errors)/rel_error_bound * 100, "%")
print("Actual error avg:", np.mean(relative_errors) , " relative to bound",  np.mean(relative_errors)/rel_error_bound * 100, "%")
#-----------------------------------------------------------------------

In [ ]:
def get_all_moments(p_in):
   #---------- fist get direct moments ----------
   M_dir = P2M_sphe(x_polar, q, p=p_in)
   #---------------------------------------------
   M_all = []
   M_trans_all = []
   x1 = np.array([0.0, 0.0, 0.0])

   for i,cx in enumerate(child_centres):
        s_pts = X[child_masks[i]]
        q_pts = q[child_masks[i]]
        s_rel = s_pts  - cx
        s_rel_polar = cart_to_sphe(s_rel)
        M = P2M_sphe(s_rel_polar, q_pts, p=p_in)
        M_trans_all.append(M2M_sphe(M, cx, x1))
        M_all.append(M)
   M_trans_all = np.array(M_trans_all) 
   M_trans = np.sum(M_trans_all, axis=0)
   return M_dir, M_trans


In [ ]:
#--------------- create list of p order and separation distances --------------
all_p = np.arange(1, 10)            
sep_dist = np.arange(2,16,2) * source_area_size
#-------------------------------------------------------------------------------

#---------- allocate arrays for results ----------
P_mom = np.zeros((len(all_p), len(sep_dist)))
P_tran = np.zeros((len(all_p), len(sep_dist)))
P_dir = np.zeros((len(all_p), len(sep_dist)))
#------------------------------------------------



#------------- loop over p and separation distances ----------
for ip, p in enumerate(all_p):
    M_dir, M_trans = get_all_moments(p)
    for jd, dist in enumerate(sep_dist):
        #------------- define center point at given distance ----------
        center = np.array([1.0, 1.0, 1.0]) * dist
        center_sphe = cart_to_sphe(np.array([center]))[0]
        #--------------------------------------------------------------
        
        P_dir[ip, jd] =  P_direct_cart(X, q, np.array([center]), eps=1e-10)[0]
        P_mom[ip, jd] = P_sphe(M_dir, np.array([center_sphe]))[0]
        P_tran[ip, jd] = P_sphe(M_trans, np.array([center_sphe]))[0]





#----------------------------------------------------------
#----------- compute errors ----------------
abs_err = np.abs(P_tran - P_dir)
with np.errstate(divide='ignore', invalid='ignore'):
    rel_err = abs_err / np.maximum(np.abs(P_dir), 1e-300)  # robust denominator
#--------------------------------------------

In [ ]:
#---------- first plot to verify parity --------------
plt.figure(figsize=(8,6))
plt.scatter(P_dir.ravel(), P_mom.ravel(), s=12, alpha=0.6)
lims = [np.nanmin(P_dir), np.nanmax(P_dir)]
plt.plot(lims, lims, lw=1)  # y=x
plt.xlabel("Direct potential")
plt.ylabel("Multipole potential")
plt.title("Parity: P_mom vs P_dir")
plt.tight_layout()
plt.show()


# --- 1) Relative error vs p, one curve per separation ---
plt.figure(figsize=(7,5))
for j, R in enumerate(sep_dist):
    plt.plot(list(all_p), rel_err[:, j], marker='o', label=f'R={R}')
plt.yscale('log')
plt.xlabel('Expansion order p')
plt.ylabel('Relative error')
plt.title('Relative error vs p (per separation)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.tight_layout()

# --- 2) Relative error vs separation, one curve per p ---
plt.figure(figsize=(7,5))
for i, p in enumerate(all_p):
    plt.plot(sep_dist, rel_err[i, :], marker='o', label=f'p={p}')
plt.xscale('log', base=2)     # you used R = 2**i; base-2 makes spacing intuitive
plt.yscale('log')
plt.xlabel('Separation R (|center|)')
plt.ylabel('Relative error')
plt.title('Relative error vs separation (per p)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(ncol=2)
plt.tight_layout()

# --- 3) Heatmap of log10 relative error over (p, R) ---
plt.figure(figsize=(6.5,5))
im = plt.imshow(
    np.log10(rel_err),          # rows = p, cols = R
    origin='lower',
    aspect='auto',
    extent=[np.log2(sep_dist[0]), np.log2(sep_dist[-1]), all_p[0], all_p[-1]]
)
cbar = plt.colorbar(im, label=r'$\log_{10}(\mathrm{relative\ error})$')
plt.xlabel(r'$\log_2 R$')
plt.ylabel('Expansion order p')
plt.title('Error heatmap over (p, R)')
plt.tight_layout()

# --- Optional: magnitude/phase sanity plots (if complex) ---
if np.iscomplexobj(P_dir):
    phase_diff = np.angle(P_mom) - np.angle(P_dir)
    phase_diff = (phase_diff + np.pi) % (2*np.pi) - np.pi  # wrap to [-pi, pi]
    plt.figure(figsize=(7,4))
    for j, R in enumerate(sep_dist):
        plt.plot(list(all_p), np.abs(phase_diff[:, j]), marker='o', label=f'R={R}')
    plt.yscale('log')
    plt.xlabel('p'); plt.ylabel('abs phase diff [rad]')
    plt.title('Phase error vs p')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.tight_layout()

plt.show()
